# Motor Characteristics

## RPM vs Duty Cycle

### Openloop Response

In [2]:
%%host

import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

from example_robot import ExampleRobot
import numpy as np
import asyncio
import stm32


class Motor(ExampleRobot):
    
    async def new_plot(self):
        spec = await super().new_plot()
        spec["columns"] = [ 'duty', 'tacho cm', 'tacho diff' ]
        return spec
    
    async def update_plot(self):
        return [ await self.get('DUTY_CM'),
                 await self.get('TACHO_CM'),
                 await self.get('TACHO_DIFF') ]

        
async def rpm_vs_duty():
    async with Motor() as robot:
        await robot.set('FS', 20)
        await robot.set('PWM_FREQ', 10_000)
        await robot.start()
        span = 20
        for duty in np.linspace(-span, span, 20):
            await robot.set('DUTY_CM', duty)
            await asyncio.sleep(0.5)

asyncio.run(rpm_vs_duty())

*** PI: unknown type: 255
MCU: start Comm @ 1000000 baud
     


### Feedforward Control

In [4]:
%%host

DEAD_ZONE = 8

def fix_duty(duty):
    if abs(duty) > 1:
        return duty+DEAD_ZONE if duty>0 else duty-DEAD_ZONE
    else:
        return 0

def unfix_duty(duty):
    if abs(duty) < DEAD_ZONE: return duty
    return duty-DEAD_ZONE if duty > 0 else duty+DEAD_ZONE


class Motor(ExampleRobot):
    
    async def new_plot(self):
        spec = await super().new_plot()
        spec["columns"] = [ 'duty', 'tacho cm', 'tacho diff' ]
        return spec
    
    async def update_plot(self):
        duty = await self.get('DUTY_CM')
        return [ unfix_duty(duty),
                 await self.get('TACHO_CM'),
                 await self.get('TACHO_DIFF') ]

async def rpm_vs_duty():
    async with Motor() as robot:
        await robot.set('FS', 20)
        await robot.set('PWM_FREQ', 10_000)
        await robot.start()
        span = 10
        for duty in np.linspace(-span, span, 20):
            duty = fix_duty(duty)
            await robot.set('DUTY_CM', duty)
            await asyncio.sleep(0.5)

asyncio.run(rpm_vs_duty())

*** PI: unknown type: 255
MCU: start Comm @ 1000000 baud
     


### Speed Controller V1

In [19]:
%%writefile $IOT_PROJECTS/robot/code/stm32/lib/speed_control.py
from robot import STATE, DUTY_CM, DUTY_DEAD, SET_DUTY_CM
from robot import Controller

class Control(Controller):
    
    def __init__(self, uart):
        super().__init__(uart)
        self.dz = STATE[DUTY_DEAD]
        
    def update(self):
        STATE[DUTY_CM] = self.fix_duty(STATE[SET_DUTY_CM])
    
    def fix_duty(self, duty):
        if abs(duty) > 0.5:
            DZ = self.dz
            return duty+DZ if duty>0 else duty-DZ
        return 0

Writing /home/iot/iot49.org/docs/projects/robot/code/stm32/lib/speed_control.py


In [21]:
%%host
import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

from example_robot import ExampleRobot
import numpy as np
import asyncio
import stm32

stm32.rsync()

class Motor(ExampleRobot):
    
    async def new_plot(self):
        spec = await super().new_plot()
        spec["columns"] = [ 'set duty', 'duty', 'tacho cm', 'tacho diff' ]
        return spec
    
    async def update_plot(self):
        return [ await self.get('SET_DUTY_CM'),
                 await self.get('DUTY_CM'),
                 await self.get('TACHO_CM'),
                 await self.get('TACHO_DIFF') ]

async def rpm_vs_duty():
    async with Motor() as robot:
        await robot.set('FS', 20)
        await robot.set('PWM_FREQ', 10_000)
        await robot.set('DUTY_DEAD', 9)
        await robot.start('speed_control')
        span = 5
        for duty in np.linspace(-span, span, 50):
            await robot.set('SET_DUTY_CM', duty)
            await asyncio.sleep(0.5)

asyncio.run(rpm_vs_duty())

Directories match
*** PI: unknown type: 255
MCU: start Comm @ 1000000 baud
     


## Step Response

Openloop time constant ~ 20ms

In [22]:
%%host

import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

from example_robot import ExampleRobot
import numpy as np
import asyncio
import stm32


class Motor(ExampleRobot):
    
    async def new_plot(self):
        spec = await super().new_plot()
        spec["columns"] = [ 'time [s]', 'tacho cm', 'tacho diff' ]
        spec["layout"] = "scatter_plot"
        spec["args"] = { 
                "title": f"Motor Step Response",
                "plot_width": 1200,
                "plot_height": 700,
            }
        return spec
    
    async def update_plot(self):
        t = await self.get('K') / await self.get('FS')
        return [ t,
                 await self.get('TACHO_CM'),
                 await self.get('TACHO_DIFF') ]

        
async def step():
    async with Motor() as robot:
        await robot.set('FS', 250)
        await robot.set('PWM_FREQ', 10_000)
        await robot.start()
        # await asyncio.sleep(0.1)
        duty = 80
        await robot.set('DUTY_CM', duty)
        await asyncio.sleep(0.15)

asyncio.run(step())

*** PI: unknown type: 255
MCU: start Comm @ 1000000 baud
     


## PID Controller

In the PID controller shown below, the "plant/process" is the controlled system (e.g. a motor). `r(t)` is the setpoint (desired value, e.g. motor rpm), `y(t)` is the actual value (e.g. motor rpm measured by an encoder), and `u(t)` is the control input (e.g. motor duty cycle). The controller attempts to minimize the error `e(t)`, i.e. make `y(t)` follow `r(t)`.

![](figures/PID_en.svg)

PID controller code:

In [6]:
%%writefile $IOT_PROJECTS/robot/code/stm32/lib/robot/PID.py
# Based on https://github.com/br3ttb/Arduino-PID-Library

from array import array

# config vector (may be dynamically updated)
SETPOINT = const(0)  # setpoint            
KP       = const(1)  # proportional term
KI       = const(2)  # scaled by /fs
KD       = const(3)  # scaled by *fs
U_MIN    = const(4)  # minimum PID output (anti-windup)
U_MAX    = const(5)  # maximum PID output (anti-windup)

# state (used internally)
_SUM     = const(6)
_Y       = const(7)

class PID:
    
    def __init__(self, config):
        self.config = config
    
    def update(self, y):
        """compute & return new PID output u from plant output y"""
        c = self.config
        err = c[SETPOINT] - y
        c[_SUM] += self._clip(c[KI] * err)   # integrator state
        u = self._clip(c[KP] * err + c[_SUM] - c[KD] * (y - c[_Y]))
        c[_Y] = y   # save last y (for KD term)
        return u
    
    def _clip(self, value):
        c = self.config
        if value > c[U_MAX]: return c[U_MAX]
        if value < c[U_MIN]: return c[U_MIN]
        return value

Writing /home/iot/iot49.org/docs/projects/robot/code/stm32/lib/robot/PID.py


## Feedback Speed Controller

Now we use a PID controller to determine the duty cycle to achieve a set speed determined by the cpt count.

We use the average cpt count for the two motors. A second PID controller will be added later to set the difference for turning.

In [6]:
%%writefile $IOT_PROJECTS/robot/code/stm32/lib/speed_control.py
from robot import STATE, DUTY_CM, DUTY_DEAD, TACHO_CM, PID_TACH_CM
from robot import Controller, PID
from robot import SETPOINT, KP

class Control(Controller):
    
    def __init__(self, uart):
        super().__init__(uart)
        self.dz  = STATE[DUTY_DEAD]
        self.pid = PID(memoryview(STATE)[PID_TACH_CM:])
        
    def update(self):
        duty = self.pid.update(STATE[TACHO_CM])
        STATE[DUTY_CM] = self.fix_duty(duty)
    
    def fix_duty(self, duty):
        if abs(duty) > 0.5:
            DZ = self.dz
            return duty+DZ if duty>0 else duty-DZ
        return 0

Writing /home/iot/iot49.org/docs/projects/robot/code/stm32/lib/speed_control.py


In [9]:
%%host

import nest_asyncio, sys, os
nest_asyncio.apply()
sys.path.append(os.path.join(os.getenv('IOT_PROJECTS'), 'robot/code/rpi'))

from example_robot import ExampleRobot
import numpy as np
import asyncio
import stm32

stm32.hard_reset()
stm32.rsync()

PID = 'TACH_CM'

class Motor(ExampleRobot):
    
    BLE_PID = 'TACH_CM'
    BLE_X   = 'DUTY_CM'
    BLE_Y   = 'DUTY_DIFF'
    
    async def ble_event(self, code: str, value: float):
        # print(f"ble_event: {code} {value:8.2f}")
        if code == 'x':
            cm = await self.get(BLE_Y)
            df = abs(100-abs(cm))
            await self.set(BLE_X, df*value)
        if code == 'y':
            await self.set(BLE_Y, 80*value)
        await super().ble_event(code, value)

    async def new_plot(self):
        spec = await super().new_plot()
        spec["columns"] = [ 'time [s]', 'set tacho', 'tacho cm', 'duty cm' ]
        spec["layout"] = "scatter_plot"
        spec["layout"] = "line_plot"
        spec["args"] = { 
                "title": f"Motor Step Response",
                "plot_width": 1200,
                "plot_height": 700,
            }
        return spec
    
    async def update_plot(self):
        t = await self.get('K') / await self.get('FS')
        return [ t,
                 await self.get_pid(PID, 'SETPOINT'),
                 await self.get('DUTY_CM'),
                 await self.get('TACHO_CM'),
                 await self.get('TACHO_DIFF') ]

        

async def step():
    async with Motor(enable_remote=True) as robot:
        robot.BLE_PID = PID
        await robot.set('FS', 100)
        await robot.set('PWM_FREQ', 10_000)
        await robot.set('DUTY_DEAD', 9)
        await robot.set_pid(PID, 'U_MIN', -100)
        await robot.set_pid(PID, 'U_MAX',  100)
        await robot.start('speed_control')
        while not robot.stop:
            kp = await robot.get_pid(PID, 'KP')
            ki = await robot.get_pid(PID, 'KI')
            kd = await robot.get_pid(PID, 'KD')
            print(f"PID: {kp:8.2f} {ki:8.2f} {kd:8.2f}")
            await robot.set_pid(PID, 'SETPOINT', 80)
            await asyncio.sleep(1)
            await robot.set_pid(PID, 'SETPOINT', 0)
            await asyncio.sleep(1)
        print("DONE")


asyncio.run(step())

Directories match
*** PI: unknown type: 255
MCU: start Comm @ 1000000 baud
     
scanning for iot49-robot
connecting to iot49-robot ... connected
PID:     0.00     0.00     0.00
PID:     0.00     0.00     0.00
PID:     0.05     0.00     0.00
PID:     0.19     0.00     0.00
PID:     0.32     0.00     0.00
PID:     0.46     0.00     0.00
PID:     0.46     0.00     0.00
PID:     0.46     0.00     0.00
PID:     0.46     0.00     0.00
PID:     0.46     0.00     0.00
PID:     0.46     0.08     0.00
PID:     0.46     0.11     0.00
PID:     0.49     0.16     0.00
PID:     0.58     0.16     0.00
PID:     0.70     0.16     0.00
PID:     0.77     0.16     0.00
PID:     0.77     0.16     0.00
PID:     0.77     0.16     0.00
PID:     0.77     0.09     0.00
PID:     0.77     0.02     0.00
PID:     0.77     0.01     0.00
PID:     0.77     0.01     0.00
PID:     0.77     0.00     0.00
PID:     0.77     0.00     0.00
PID:     0.77     0.00     0.00
PID:     0.77     0.00     0.00
PID:     0.77     0.00

In [41]:
%%service host
systemctl restart hciuart.service